In [582]:


import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# view all columns
pd.set_option('display.max_columns', None)
#false positive warning, will not display
pd.options.mode.chained_assignment = None  # default='warn'

## Importing and Cleaning Monthly Data

In [572]:
# import monthly data
# Monthly data grouped by year from the OCC website: previous Python script iterated through to combine
# the data into one file, summed months for a yearly volume in bbls
monthly_all = pd.read_csv("../oklahoma_earthquakes_largefiles/occ_monthly_cleaned/occ_monthly_cleaned.csv", low_memory=False)

In [573]:
# view monthly data
# monthly_all.head()

In [574]:
# drop columns
monthly_all.drop(columns = ["JanPSI", "FebPSI", "MarPSI", "AprPSI", "MayPSI", "JunPSI", "JulPSI", "AugPSI", "SepPSI", "OctPSI", "NovPSI", "DecPSI"], inplace = True)
monthly_all.drop(columns = ["Township", "Range", "QTR4", "QTR3", "QTR2", "QTR1", "PM", "FluidType"], inplace=True)
monthly_all.drop(columns = ["ProdReportType","CommissionOrderNo", "Status", "WellStatus", "WellType"], inplace = True)
monthly_all.drop(columns = ["Section", "ModifyDate", "Packer", "PackerDepth", "ReportYear", "MeasurementType"], inplace = True)
monthly_all.drop(columns = ["LastMITDate", "InjTopDepth", "InjBotDepth", "PlugBackTotalDepth"], inplace = True)

In [575]:
# Renaming column for consistency, multiple lines for readability

monthly_all.rename(columns={"OperatorName": "Operator_Name", "WellName": "Well_Name", "WellNumber":"Well_Number"}, inplace = True)
monthly_all.rename(columns={"LAT": "Latitude", "LON": "Longitude", "FormationName":"Formation_Name"}, inplace = True)
monthly_all.rename(columns={"JanVol": "Jan_Vol", "FebVol": "Feb_Vol", "MarVol":"Mar_vol", "AprVol":"Apr_Vol"}, inplace = True)
monthly_all.rename(columns={"MayVol": "May_Vol", "JunVol": "Jun_Vol", "JulVol":"Jul_Vol", "AugVol":"Aug_Vol"}, inplace = True)
monthly_all.rename(columns={"SepVol": "Sep_Vol", "OctVol": "Oct_Vol", "NovVol":"Nov_Vol", "DecVol":"Dec_Vol"}, inplace = True)
monthly_all.rename(columns={"TotalDepth": "Total_Depth"}, inplace = True)

In [576]:
# Add data origin column (monthly vs daily); will be less obvious in the final master file
monthly_all["data_origin"] = "monthly"

In [577]:
# Add columns to match daily export df
monthly_all["Operator_Number"] = ""

In [578]:
# Reorder Columns
monthly_all = monthly_all[['API', 'API_year', 'year', 'Operator_Name', 'Operator_Number',
       'Well_Name', 'Well_Number', 'Latitude', 'Longitude', 'County',
       'Total_Depth', 'Formation_Name', 'Jan_Vol', 'Feb_Vol', 'Mar_vol',
       'Apr_Vol', 'May_Vol', 'Jun_Vol', 'Jul_Vol', 'Aug_Vol', 'Sep_Vol',
       'Oct_Vol', 'Nov_Vol', 'Dec_Vol', 'year_volume', 'data_origin']]

In [520]:
# # Get number of unque wells per year for Arbuckle wells

# # get list of years and sort ascending
# years = list(monthly_all.year.unique())
# years.sort()
# # count of total wells per year
# well_count = []
# # count of wells completed in Arbuckle Formation
# well_count_arb = []


# for year in years:
#     monthly_temp = monthly[monthly_all.year == year]
#     n = len(pd.unique(monthly_temp['API']))
#     well_count.append(n)
#     monthly_temp = monthly_temp[monthly_temp.FormationName == "ARBUCKLE"]
#     n = len(pd.unique(monthly_temp['API']))
#     well_count_arb.append(n)

# well_count_dict = dict(zip(years, well_count))
# well_count_arb_dict = dict(zip(years, well_count_arb))
# print(well_count_dict, well_count_arb_dict)

{2011: 10386, 2012: 10902, 2013: 11004, 2014: 11277, 2015: 10880, 2016: 11087, 2017: 11203, 2018: 11124, 2019: 11032, 2020: 10839} {2011: 205, 2012: 316, 2013: 396, 2014: 462, 2015: 437, 2016: 459, 2017: 472, 2018: 457, 2019: 453, 2020: 448}


## Importing and Cleaning Daily Data; Getting monthly volumes for each Well

In [564]:
# import test daily injection data, grouped by year

daily = pd.read_csv("../oklahoma_earthquakes_largefiles/occ_daily_injection_cleaned/arb_2014_cleaned.csv", low_memory=False)

In [565]:
# view data
daily.dtypes

API                        float64
Well_Name                   object
Well_Number                 object
Operator_Number            float64
Operator_Name               object
Daily_Report_Date_Start     object
Daily_Report_Date_End       object
Report_Date                 object
Volume_BPD                 float64
Pressure_PSI               float64
Latitude                   float64
Longitude                  float64
DirArea                     object
Directive_Status            object
dtype: object

In [568]:
daily.API = daily.API.round(0).astype(int)

In [581]:
daily.head(31)

,API,Well_Name,Well_Number,Operator_Number,Operator_Name,Daily_Report_Date_Start,Daily_Report_Date_End,Report_Date,Volume_BPD,Pressure_PSI,Latitude,Longitude,DirArea,Directive_Status
0,3500322020,ROSE KELLY SWD,1-3,20357.0,SANDRIDGE EXPLORATION & PRODUCTION LLC,2014-11-23,2020-03-29,2014-12-31,3246.0,0.0,36.928442,-98.256115,OWRA,NOT IN GRANITE
1,3500322020,ROSE KELLY SWD,1-3,20357.0,SANDRIDGE EXPLORATION & PRODUCTION LLC,2014-11-23,2020-03-29,2014-12-30,3373.0,0.0,36.928442,-98.256115,OWRA,NOT IN GRANITE
2,3500322020,ROSE KELLY SWD,1-3,20357.0,SANDRIDGE EXPLORATION & PRODUCTION LLC,2014-11-23,2020-03-29,2014-12-29,3508.0,0.0,36.928442,-98.256115,OWRA,NOT IN GRANITE
3,3500322020,ROSE KELLY SWD,1-3,20357.0,SANDRIDGE EXPLORATION & PRODUCTION LLC,2014-11-23,2020-03-29,2014-12-28,3363.0,0.0,36.928442,-98.256115,OWRA,NOT IN GRANITE
4,3500322020,ROSE KELLY SWD,1-3,20357.0,SANDRIDGE EXPLORATION & PRODUCTION LLC,2014-11-23,2020-03-29,2014-12-27,3226.0,0.0,36.928442,-98.256115,OWRA,NOT IN GRANITE
5,3500322020,ROSE KELLY SWD,1-3,20357.0,SANDRIDGE EXPLORATION & PRODUCTION LLC,2014-11-23,2020-03-29,2014-12-26,2799.0,0.0,36.928442,-98.256115,OWRA,NOT IN GRANITE
6,3500322020,ROSE KELLY SWD,1-3,20357.0,SANDRIDGE EXPLORATION & PRODUCTION LLC,2014-11-23,2020-03-29,2014-12-25,4363.0,0.0,36.928442,-98.256115,OWRA,NOT IN GRANITE
7,3500322020,ROSE KELLY SWD,1-3,20357.0,SANDRIDGE EXPLORATION & PRODUCTION LLC,2014-11-23,2020-03-29,2014-12-24,3865.0,0.0,36.928442,-98.256115,OWRA,NOT IN GRANITE
8,3500322020,ROSE KELLY SWD,1-3,20357.0,SANDRIDGE EXPLORATION & PRODUCTION LLC,2014-11-23,2020-03-29,2014-12-23,3527.0,0.0,36.928442,-98.256115,OWRA,NOT IN GRANITE
9,3500322020,ROSE KELLY SWD,1-3,20357.0,SANDRIDGE EXPLORATION & PRODUCTION LLC,2014-11-23,2020-03-29,2014-12-22,2864.0,0.0,36.928442,-98.256115,OWRA,NOT IN GRANITE


In [523]:
# add api_year_month column
# will be unique column to group APIs and return monthly volumes for each API

daily["API_year_month"] = daily.API.astype(str).str[:] + "-" + daily.Report_Date.str[0:7]

# create API_year column for unique identification for the export DF later
daily["API_year"] = daily.API.astype(str).str[:] + "-" + daily.Report_Date.str[0:4]
# daily

In [524]:
# Get the monthly injection volumes for each well using the new unique API_year_month column

# group by API-year-month
daily_grouped = daily.groupby(["API_year_month"]).sum()
# select just the volume
daily_grouped = daily_grouped["Volume_BPD"]
# convert to dictionary; will be searched later
daily_grouped_dict = daily_grouped.to_dict()

In [525]:
# view grouped data
# daily_grouped

In [526]:
# create dataframe of just unique wells/API; will be the start of export dataframe
# dataframe of one well per year
daily_export = daily.drop_duplicates(subset="API")

In [527]:
# Drop unnecessary columns from new DF
daily_export.drop(columns = ["Daily_Report_Date_Start", "Daily_Report_Date_End", "Volume_BPD", "Pressure_PSI", "DirArea", "Directive_Status", "API_year_month", "Report_Date"], inplace = True)

In [528]:
# creating list of new columns for new DF
new_col_list = "Jan_Vol, Feb_Vol, Mar_Vol, Apr_Vol, May_Vol, Jun_Vol, Jul_Vol, Aug_Vol, Sep_Vol, Oct_Vol, Nov_Vol, Dec_Vol, year"
new_col_list = new_col_list.split(", ")

In [529]:
# adding new colums to new DF to match monthly 
for i in new_col_list:
    daily_export[i] = ''

In [530]:
# adding year and months values to cells

daily_export["year"] = daily_export.apply(
    lambda row: row.API_year[11:15], axis = 1)

In [531]:
# define function to populate columns with monthly data

monthly_volume_cols = ['Jan_Vol', 'Feb_Vol', 'Mar_Vol', 'Apr_Vol', 'May_Vol', 'Jun_Vol', 'Jul_Vol', 'Aug_Vol', 'Sep_Vol', 'Oct_Vol', 'Nov_Vol', 'Dec_Vol']

month_dict = {"Jan_Vol": "01", "Feb_Vol": "02", "Mar_Vol": "03", "Apr_Vol": "04", 
              "May_Vol": "05", "Jun_Vol": "06", "Jul_Vol": "07", "Aug_Vol": "08", 
              "Sep_Vol": "09", "Oct_Vol": "10", "Nov_Vol": "11", "Dec_Vol": "12"}

def populate_values(row):
    try:
        result = daily_grouped_dict[(str(row.API) + "-" + row.year+ "-" + month_dict[col])]
        return result
#   if the key is not found in the dictionary containing values for each API_month, NaN is returned
    except:
        return np.nan

In [532]:
# Apply above function to each column
for col in monthly_volume_cols:
    daily_export[col] = daily_export.apply(populate_values, axis = 1)

In [533]:
daily_export.fillna(0, inplace = True)

In [534]:
# Calcalate and add column for yearly volume
daily_export["year_volume"] = daily_export.apply(
    lambda row: row.Jan_Vol + row.Feb_Vol + row.Mar_Vol + row.Apr_Vol + row.May_Vol + row.Jun_Vol + row.Jul_Vol + row.Aug_Vol + row.Sep_Vol + row.Oct_Vol + row.Nov_Vol + row.Dec_Vol, axis = 1)

In [535]:
# Add data origin column (monthly vs daily); will be less obvious in the final master file
daily_export["data_origin"] = "daily"

In [536]:
# Add columns to match monthly df
daily_export["County"] = ""
daily_export["Total_Depth"] = ""
daily_export["Formation_Name"] = "Arbuckle"

In [537]:
# Reorder Columns
daily_export =daily_export[['API', 'API_year', 'year', 'Operator_Name', 'Operator_Number',
       'Well_Name', 'Well_Number', 'Latitude', 'Longitude', 'County',
       'Total_Depth', 'Formation_Name', 'Jan_Vol', 'Feb_Vol', 'Mar_Vol',
       'Apr_Vol', 'May_Vol', 'Jun_Vol', 'Jul_Vol', 'Aug_Vol', 'Sep_Vol',
       'Oct_Vol', 'Nov_Vol', 'Dec_Vol', 'year_volume', 'data_origin']]

In [570]:
daily_export.dtypes

API                  int64
API_year            object
year                object
Operator_Name       object
Operator_Number      int64
Well_Name           object
Well_Number         object
Latitude           float64
Longitude          float64
County              object
Total_Depth         object
Formation_Name      object
Jan_Vol            float64
Feb_Vol            float64
Mar_Vol            float64
Apr_Vol            float64
May_Vol            float64
Jun_Vol            float64
Jul_Vol            float64
Aug_Vol            float64
Sep_Vol            float64
Oct_Vol            float64
Nov_Vol            float64
Dec_Vol            float64
year_volume        float64
data_origin         object
dtype: object